In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score,mean_squared_error

In [ ]:
df_train=pd.read_csv('train.csv')

In [ ]:
print(f"train data: Rows={df_train.shape[0]},Columns={df_train.shape[1]}")

train data: Rows=29451,Columns=13


In [ ]:
df_train.head()

,POSTED_BY,UNDER_CONSTRUCTION,RERA,BHK_NO.,BHK_OR_RK,SQUARE_FT,READY_TO_MOVE,RESALE,ADDRESS,LONGITUDE,LATITUDE,TARGET(PRICE_IN_LACS)
0,Owner,0,0,2,BHK,1300.236407,1,1,"Ksfc Layout,Bangalore",12.969910,77.597960,55.0
1,Dealer,0,0,2,BHK,1275.000000,1,1,"Vishweshwara Nagar,Mysore",12.274538,76.644605,51.0
2,Owner,0,0,2,BHK,933.159722,1,1,"Jigani,Bangalore",12.778033,77.632191,43.0
3,Owner,0,1,2,BHK,929.921143,1,1,"Sector-1 Vaishali,Ghaziabad",28.642300,77.344500,62.5
4,Dealer,1,0,2,BHK,999.009247,0,1,"New Town,Kolkata",22.592200,88.484911,60.5


In [ ]:
df_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29451 entries, 0 to 29450
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   POSTED_BY              29451 non-null  object 
 1   UNDER_CONSTRUCTION     29451 non-null  int64  
 2   RERA                   29451 non-null  int64  
 3   BHK_NO.                29451 non-null  int64  
 4   BHK_OR_RK              29451 non-null  object 
 5   SQUARE_FT              29451 non-null  float64
 6   READY_TO_MOVE          29451 non-null  int64  
 7   RESALE                 29451 non-null  int64  
 8   ADDRESS                29451 non-null  object 
 9   LONGITUDE              29451 non-null  float64
 10  LATITUDE               29451 non-null  float64
 11  TARGET(PRICE_IN_LACS)  29451 non-null  float64
dtypes: float64(4), int64(5), object(3)
memory usage: 2.7+ MB


In [ ]:
df_train['CITY']=df_train['ADDRESS'].apply(lambda x: x.split(',')[-1])
df_train['CITY']

0        Bangalore
1           Mysore
2        Bangalore
3        Ghaziabad
4          Kolkata
           ...    
29446         Agra
29447         Vapi
29448       Jaipur
29449      Chennai
29450       Jaipur
Name: CITY, Length: 29451, dtype: object

In [ ]:
train=df_train.drop(['ADDRESS','LONGITUDE','LATITUDE'],axis=1)

In [ ]:
def map_city(city):
  if city in ['Ahmedabad', 'Bangalore', 'Chennai', 'Delhi', 'Hyderabad',
   'Kolkata', 'Mumbai', 'Pune', 'Maharashtra']:
    return 'tier1'
  elif city in ['Agra', 'Ajmer', 'Aligarh', 'Amravati', 'Amritsar', 'Asansol',
    'Aurangabad', 'Bareilly','Belgaum', 'Bhavnagar', 'Bhiwandi', 'Bhopal', 'Bhubaneswar','Bikaner', 'Bilaspur', 'Bokaro Steel City',
    'Chandigarh', 'Coimbatore', 'Cuttack', 'Dehradun', 'Dhanbad','Bhilai', 'Durgapur', 'Dindigul', 'Erode','Faridabad', 'Firozabad', 'Ghaziabad', 'Gorakhpur', 'Gulbarga',
    'Guntur', 'Gwalior', 'Gurgaon', 'Guwahati','Hamirpur', 'Hubli–Dharwad', 'Indore', 'Jabalpur', 'Jaipur','Jalandhar', 'Jammu', 'Jamnagar', 'Jamshedpur',
    'Jhansi', 'Jodhpur', 'Kakinada', 'Kannur', 'Kanpur', 'Karnal','Kochi', 'Kolhapur', 'Kollam', 'Kozhikode''Kurnool', 'Ludhiana', 'Lucknow', 'Madurai', 'Malappuram',
    'Mathura', 'Mangalore', 'Meerut', 'Moradabad','Mysore', 'Nagpur', 'Nanded', 'Nashik', 'Nellore', 'Noida','Patna', 'Pondicherry', 'Purulia', 'Prayagra'
    'Raipur', 'Rajkot', 'Rajahmundry', 'Ranchi', 'Rourkela', 'Ratlam','Salem', 'Sangli', 'Shimla', 'Siliguri','Solapur', 'Srinagar', 'Surat', 'Thanjavur',
    'Thiruvananthapuram', 'Thrissur', 'Tiruchirappalli', 'Tirunelveli','Tiruvannamalai', 'Ujjain', 'Bijapur', 'Vadodara', 'Varanasi',
    'Vasai-Virar City', 'Vijayawada', 'Visakhapatnam','Vellore', 'Warangal']:
    return 'tier2'
  else:
    return 'tier3'

In [ ]:
train['CITY']=train['CITY'].apply(map_city)

In [ ]:
train.head()

,POSTED_BY,UNDER_CONSTRUCTION,RERA,BHK_NO.,BHK_OR_RK,SQUARE_FT,READY_TO_MOVE,RESALE,TARGET(PRICE_IN_LACS),CITY
0,Owner,0,0,2,BHK,1300.236407,1,1,55.0,tier1
1,Dealer,0,0,2,BHK,1275.000000,1,1,51.0,tier2
2,Owner,0,0,2,BHK,933.159722,1,1,43.0,tier1
3,Owner,0,1,2,BHK,929.921143,1,1,62.5,tier2
4,Dealer,1,0,2,BHK,999.009247,0,1,60.5,tier1


In [ ]:
label_encoder=preprocessing.LabelEncoder()

In [ ]:
train['POSTED_BY']=label_encoder.fit_transform(train['POSTED_BY'])
train['CITY']=label_encoder.fit_transform(train['CITY'])
train['BHK_OR_RK']=label_encoder.fit_transform(train['BHK_OR_RK'])
train.head()

,POSTED_BY,UNDER_CONSTRUCTION,RERA,BHK_NO.,BHK_OR_RK,SQUARE_FT,READY_TO_MOVE,RESALE,TARGET(PRICE_IN_LACS),CITY
0,2,0,0,2,0,1300.236407,1,1,55.0,0
1,1,0,0,2,0,1275.000000,1,1,51.0,1
2,2,0,0,2,0,933.159722,1,1,43.0,0
3,2,0,1,2,0,929.921143,1,1,62.5,1
4,1,1,0,2,0,999.009247,0,1,60.5,0


In [ ]:
train['SQUARE_FT']=np.log(train['SQUARE_FT'])
train['SQUARE_FT'].head()

0    7.170301
1    7.150701
2    6.838576
3    6.835100
4    6.906764
Name: SQUARE_FT, dtype: float64

In [ ]:
X=train.drop(['TARGET(PRICE_IN_LACS)'],axis=1)
y=train['TARGET(PRICE_IN_LACS)']

In [ ]:
y.head()

0    55.0
1    51.0
2    43.0
3    62.5
4    60.5
Name: TARGET(PRICE_IN_LACS), dtype: float64

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=100)

In [ ]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(20615, 9)
(20615,)
(8836, 9)
(8836,)


In [ ]:
#Linear Regression
lr=LinearRegression()
lr.fit(X_train,y_train)
y_pred=lr.predict(X_test)
y_pred

array([-470.1875 ,   46.5    ,  148.5625 , ...,  -56.125  ,  -14.40625,
       -462.0625 ])

In [ ]:
r2=r2_score(y_test,y_pred)
print('Accuracy of the linear regression model (R-Square value):',r2*100)

Accuracy of the linear regression model (R-Square value): 52.427054951569694


In [ ]:
#Random Forest Regressor
rf_model=RandomForestRegressor(n_estimators=100,random_state=80)
rf_model.fit(X_train,y_train)
y_pred=rf_model.predict(X_test)
y_pred

array([  79.786     , 1068.05105   ,   94.14      , ...,   31.49      ,
         53.598     ,   44.13671429])

In [ ]:
r2=r2_score(y_test,y_pred)
print('Accuracy of the random forest regressor model (R-Square value):',r2*100)

Accuracy of the random forest regressor model (R-Square value): 92.62647402919805


In [ ]:
#Decision Tree Regressor
dt=DecisionTreeRegressor()
dt.fit(X_train,y_train)
y_pred=dt.predict(X_test)
y_pred

array([  80., 1700.,  100., ...,   27.,   55.,   14.])

In [ ]:
r2=r2_score(y_test,y_pred)
print('Accuracy of the linear regression model (R-Square value):',r2*100)

Accuracy of the linear regression model (R-Square value): 86.08425333113327
